### Import Needed Packages

In [ ]:
import os

from __future__ import print_function
import requests

from fsspec.implementations.http import HTTPFileSystem
import plotly.express as px
import xarray as xr

import basd

# Precipitation Data

Here we will adjust precipitation output from NCAR's CESM2-WACCM model using observational data from EWEMBI.

First, we define a function created by an unknown author which produces OpenDAP url's for a given CMIP6 query using ESGF

In [ ]:
# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing

# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = []
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    return sorted(all_files)

Here we query for daily precipitation data from NCAR's CESM2-WACCM model over the historical time period, and the 'r1i1p1f1' variant.

In [ ]:
sim_hist_result = esgf_search(activity_id='CMIP', variable_id='pr', experiment_id='historical', frequency='day',
                  institution_id="NCAR", source_id="CESM2-WACCM", member_id="r1i1p1f1")

Here we open the data from all of the returning files using `xarray.open_mfdataset()` which will use Dask arrays as the backend, chunking every 365th timestep.

In [ ]:
pr_sim_hist = xr.open_mfdataset(sim_hist_result[-17:], chunks={'time': 365})

Here we query for daily precipitation data from NCAR's CESM2-WACCM model over the future time period under SSP 2 and RCP 4.5, and the 'r1i1p1f1' variant.

In [ ]:
sim_fut_result = esgf_search(activity_id='ScenarioMIP', variable_id='pr', experiment_id='ssp245', frequency='day',
                  institution_id="NCAR", source_id="CESM2-WACCM", member_id="r1i1p1f1")

Here we open the data from all of the returning files using `xarray.open_mfdataset()` which will use Dask arrays as the backend, chunking every 365th timestep.

In [ ]:
pr_sim_fut = xr.open_mfdataset(sim_fut_result[-9:], chunks={'time': 365})

Now we read in the observational data, from GSWP3. This data isn't on OpenDAP, so we use `fsspec` to get around that.

In [ ]:
url_list = ['https://files.isimip.org/ISIMIP2a/InputData/climate_co2/climate/HistObs/GSWP3/pr_gswp3_1971_1980.nc4',
'https://files.isimip.org/ISIMIP2a/InputData/climate_co2/climate/HistObs/GSWP3/pr_gswp3_1981_1990.nc4',
'https://files.isimip.org/ISIMIP2a/InputData/climate_co2/climate/HistObs/GSWP3/pr_gswp3_1991_2000.nc4',
'https://files.isimip.org/ISIMIP2a/InputData/climate_co2/climate/HistObs/GSWP3/pr_gswp3_2001_2010.nc4']
fs = HTTPFileSystem()
fobjs = [fs.open(url) for url in url_list]

In [ ]:
pr_obs_hist = xr.open_mfdataset(fobjs, chunks={'time': 365})

## Setting parameters and creating Adjustment Object
For precipitation, we set the following parameters:

* Lower bound: 0
* Lower threshold: 0.0000011547
* Trend preservation method: 'mixed'
* Distribution: 'gamma'
* Value to set cells with only invalid values: 0

We then create the bias adjustment object with our data, parameters object, and ask to have our data remapped to match in resolution. This is needed in this case as the EWEMBI data is 360x720 (lat x lon), where CESM2-WACCM is 192x288.

In [ ]:
pr_sim_hist

In [ ]:
params = basd.Parameters(lower_bound=0,
                         lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma',
                         if_all_invalid_use=0)
pr_ba = basd.Adjustment(pr_obs_hist,
                        pr_sim_hist,
                        pr_sim_fut,
                        'pr',
                        params,
                        remap_grid=True)

# Adjustment at one location
Here we ask to perform bias adjustment at the 100th row and 100th column cell position. This happens to correspond to 4.421 degrees latitude (on -90 to 90) and 125 degrees longitude (on -180 to 180). This choice was arbitrary.

Note that to run a full grid adjustment, one would use `pr_ba.adjust_bias()`. However, this is extremely computationally extensive and is recommended to be run on computing cluster to make use of the parallel implementation of this function.

In [ ]:
pr_sim_fut_ba_loc = pr_ba.adjust_bias_one_location(dict(lat=100, lon=100))

# Plots

### Histogram
This plot shows the distribution of precipitation over the time period Jan 1, 2095 - Dec 31, 2100 (our input simulated future period) at our given grid cell, before and after bias adjustment.

In [ ]:
pr_sim_fut_ba_loc.plot_hist()

### Empirical CDF
This plot gives the empirical CDFs of precipitation for each of our input data, and bias adjusted result, at the given grid cell.

We can see here how the relationship between the observational and simulated historical data, is indeed transferred from the simulated future data to the bias adjusted result.

In [ ]:
pr_sim_fut_ba_loc.plot_ecdf(log_x=True)

# Shortwave Radiation Data

Now we will adjust surface downwelling shortwave radiation output from MIROC's model using observational data from EWEMBI.

Again we set the paths to our data:

In [ ]:
rsds_obs_hist_path = 'rsds_ewembi_2011_2016.nc4'
rsds_sim_hist_path = 'rsds_day_MIROC6_historical_r2i1p1f1_gn_20100101-20141231.nc'
rsds_sim_fut_path = 'rsds_day_MIROC6_ssp370_r2i1p1f1_gn_20150101-20241231.nc'

reading in with `xarray`:

In [ ]:
rsds_obs_hist = xr.open_dataset(os.path.join(data_path, rsds_obs_hist_path),
                                decode_coords = 'all')
rsds_sim_hist = xr.open_dataset(os.path.join(data_path, rsds_sim_hist_path),
                                decode_coords = 'all')
rsds_sim_fut = xr.open_dataset(os.path.join(data_path, rsds_sim_fut_path),
                               decode_coords = 'all')

## Creating Parameter and Adjustment Objects
For `rsds` we have a bit more involved process to set up the bias adjustment, though that is all taken care of for us when we specify parameters. This is because we are going to first scale `rsds` to the interval [0,1], at which point is assumed to follow a Beta distribution. We'll then set our remaining parameters accordingly.

Our data is scaled to [0,1] by setting each observation to be how large the observation is compared to the largest observation in a surrounding window. We get to choose how large the window by specifying the half width (so number of days just before or after). Here we set a half running window size of 15, thus a full window size of 31.

Again, we also want to remap the observational data to match the simulated data's resolution.


In [ ]:
rsds_params = basd.Parameters(halfwin_ubc=15,
                              trend_preservation='bounded',
                              distribution='beta',
                              lower_bound=0,
                              upper_bound=1,
                              lower_threshold=0.0001,
                              upper_threshold=0.9999,
                              if_all_invalid_use=0)
rsds_ba = basd.Adjustment(rsds_obs_hist,
                          rsds_sim_hist,
                          rsds_sim_fut,
                          'rsds',
                          rsds_params,
                          remap_grid=True)

# Adjustment at one location
Here we ask to perform bias adjustment at the 100th row and 100th column cell position. This time this happens to correspond to 51.1 degrees latitude and 140.6 degrees longitude. Again, arbitrarily selected.

In [ ]:
rsds_sim_fut_ba_loc = rsds_ba.adjust_bias_one_location(dict(lat=100, lon=100))

# Plots II

### Histogram
Shows the distribution of surface downwelling shortwave radiation before and after bias adjustment at the chosen grid cell.

In [ ]:
rsds_sim_fut_ba_loc.plot_hist()

### Empirical CDF
Shows the empirical CDFs for each input data source and resulting adjustment time series, at the chosen grid cell.

In [ ]:
px.ecdf(rsds_sim_fut_ba_loc.time_series, x='rsds', color='Source')